In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../..")

import numpy as np
import matplotlib.pyplot as plt
import os
from src import data
import json
from tqdm.auto import tqdm

In [ ]:
from src import models, data, functional

###############################################
relation_name = "person university"
###############################################

In [ ]:
mt = models.load_model(name="gptj", fp16=True, device="cuda")

In [ ]:
dataset = data.load_dataset()
relation = dataset.filter(
    relation_names=[relation_name]
)[0]

len(relation.samples)

In [ ]:
train, test = relation.split(6)
icl_prompt = functional.make_prompt(
    mt=mt,
    prompt_template= " {} :",
    subject="{}",
    examples=train.samples,
)

In [ ]:
test_relation = (
    functional.filter_relation_samples_based_on_provided_fewshots(
        mt=mt,
        test_relation=test,
        prompt_template=icl_prompt,
    )
)

In [ ]:
f"{len(test_relation.samples)}/{len(relation.samples) - len(train.samples)}"

In [ ]:
subj_to_obj = {sample.subject: sample.object for sample in relation.samples}

for query in os.listdir(relation_name.replace(" ", "_")):
    query_path = os.path.join(relation_name.replace(" ", "_"), query)
    with open(query_path, "r") as f:
        query_result = json.load(f)
    
    for const_info in query_result:
        star = const_info["subjectLabel"]
        constellation = const_info["objectLabel"]
        # if star.startswith("HR") or star.startswith("HD"):
        #     continue
        if "University" not in constellation:
            continue
        if star in subj_to_obj:
            subj_to_obj.pop(star)
        subj_to_obj[star] = constellation

In [ ]:
relation.samples

In [ ]:
candidate_samples = [
    data.RelationSample(
        subject=star,
        object = constellation
    ) for star, constellation in subj_to_obj.items()
]

In [ ]:
import torch
with torch.no_grad():
    correct_prediction = (
        functional.filter_relation_samples_based_on_provided_fewshots(
            mt=mt,
            test_relation=relation.set(samples=candidate_samples[:1000]),
            prompt_template=icl_prompt,
            batch_size=4
        )
    )

In [ ]:
f"{len(correct_prediction.samples)} / {len(candidate_samples)}"

In [ ]:
with open(f"{relation_name.lower().replace(' ', '_')}.json", "w") as f:
    json.dump(relation.set(samples = candidate_samples[:1000]).to_dict(), f)

In [ ]:
correct_prediction.samples